In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'flax-community/papuGaPT2'
device = 'cuda'
device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label


def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

sentences = [
  'To jest zwykłe polskie zdanie.',
  'This is a normal English sentence.',
  'iweryuiiu hrfw3eieur fr'
]

print ()
for s in sentences:
    print (s, sentence_prob(s))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/864 [00:01<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]


To jest zwykłe polskie zdanie. -26.38755
This is a normal English sentence. -37.866207
iweryuiiu hrfw3eieur fr -88.53769


In [ ]:
def positive_or_negative(sentence_text):
  pos_prob=sentence_prob(sentence_text+"bardzo polecam")
  neg_prob=sentence_prob(sentence_text+"nie polecam")
  if(neg_prob>pos_prob):
    return "BAD"
  else:
    return "GOOD"


In [ ]:
negatywne=['W wielu pokojach niedziałająca klimatyzacja.',
'Jedzenie mimo rzekomych dni europejskich monotonne.',
'Drożej niż u konkurencji w podobnym standardzie.',
'Może za szybko zrezygnowałam, ale szkoda mi było wydawać pieniędzy na spotkania, które nie przynosiły efektu.','Omijaj to miejsce!']

for i in negatywne:
  print(positive_or_negative(i))



BAD
BAD
BAD
BAD
BAD


In [ ]:
pozytywne=['Parking monitorowany w cenie.',
'Hotel czysty, pokoje były sprzątane bardzo dokłądnie.',
'Generalnie mogę go polecić, kierował mnie na potrzebne badania, analizował ich wyniki, cierpliwie odpowiadał na pytania.',
'Fajny klimat pofabrykanckich kamienic.',
'Sala zabaw dla dzieci, plac zabaw na zewnątrz, kominek, tenis stołowy.']

for i in pozytywne:
  print(positive_or_negative(i))

GOOD
GOOD
GOOD
GOOD
GOOD


In [ ]:
with open('/content/drive/My Drive/chiny_kicza/reviews_for_task3.txt', 'r') as f:
  lines = f.readlines()
  i=0
  positive_lines_list=[]
  negative_lines_list=[]
  for line in lines:
    words=line.split()
    if (words[0]=='GOOD'):
      positive_lines_list.append(' '.join(words[1:]))
    else:
      negative_lines_list.append(' '.join(words[1:]))

n=20
cnt=0

for i in random.sample(positive_lines_list, n):
  #print(i,positive_or_negative(i))
  if (positive_or_negative(i)=='GOOD'):
    cnt+=1
print('Pozytywne',cnt/n)

n=50
cnt=0

for i in random.sample(negative_lines_list, n):
  #print(i,positive_or_negative(i))
  if (positive_or_negative(i)=='BAD'):
    cnt+=1
print('Negatywne',cnt/n)

Pozytywne 0.85
Negatywne 0.86


In [ ]:
def positive_or_negative(sentence_text):
  pos_prob=sentence_prob(sentence_text+"To była pozytywna opinia.")
  neg_prob=sentence_prob(sentence_text+"To była negatywna opnia.")
  if(neg_prob>pos_prob):
    return "BAD"
  else:
    return "GOOD"

In [ ]:
with open('/content/drive/My Drive/chiny_kicza/reviews_for_task3.txt', 'r') as f:
  lines = f.readlines()
  i=0
  positive_lines_list=[]
  negative_lines_list=[]
  for line in lines:
    words=line.split()
    if (words[0]=='GOOD'):
      positive_lines_list.append(' '.join(words[1:]))
    else:
      negative_lines_list.append(' '.join(words[1:]))

n=20
cnt=0

for i in random.sample(positive_lines_list, n):
  #print(i,positive_or_negative(i))
  if (positive_or_negative(i)=='GOOD'):
    cnt+=1
print('Pozytywne',cnt/n)

n=50
cnt=0

for i in random.sample(negative_lines_list, n):
  #print(i,positive_or_negative(i))
  if (positive_or_negative(i)=='BAD'):
    cnt+=1
print('Negatywne',cnt/n)

Pozytywne 1.0
Negatywne 0.0


In [ ]:
def positive_or_negative(sentence_text):
  pos_prob=sentence_prob(sentence_text+"Wspaniale")
  neg_prob=sentence_prob(sentence_text+"Tragedia")
  if(neg_prob>pos_prob):
    return "BAD"
  else:
    return "GOOD"

In [ ]:
with open('/content/drive/My Drive/chiny_kicza/reviews_for_task3.txt', 'r') as f:
  lines = f.readlines()
  i=0
  positive_lines_list=[]
  negative_lines_list=[]
  for line in lines:
    words=line.split()
    if (words[0]=='GOOD'):
      positive_lines_list.append(' '.join(words[1:]))
    else:
      negative_lines_list.append(' '.join(words[1:]))

n=20
cnt=0

for i in random.sample(positive_lines_list, n):
  #print(i,positive_or_negative(i))
  if (positive_or_negative(i)=='GOOD'):
    cnt+=1
print('Pozytywne',cnt/n)

n=50
cnt=0

for i in random.sample(negative_lines_list, n):
  #print(i,positive_or_negative(i))
  if (positive_or_negative(i)=='BAD'):
    cnt+=1
print('Negatywne',cnt/n)

Pozytywne 0.85
Negatywne 0.58
